In [2]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_dir = os.path.split(os.getcwd())[0]
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

In [6]:

import republic.parser.republic_column_parser as column_parser
from elasticsearch import Elasticsearch
import republic.parser.republic_page_parser as page_parser
import republic.parser.republic_paragraph_parser as paragraph_parser
import republic.parser.republic_file_parser as file_parser

import copy



In [7]:
from republic.config.republic_config import base_config, set_config_inventory_num

inventory_num = 3767
data_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr"

inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)
print(inventory_config)
scan_files = file_parser.get_files(inventory_config["data_dir"])
print("Number of scan files:", len(scan_files))

scan_file = scan_files[10]
print(scan_file)


# scan num 751 is index start
# scan num 6 is resolution start

scan_file = scan_files[5]
scan_hocr = page_parser.get_scan_hocr(scan_file, inventory_config)
#print(scan_hocr["scan_type"])
#print(scan_hocr["page_box"])
if "double_page" in scan_hocr["scan_type"]:
    pages_hocr = page_parser.parse_double_page_scan(scan_hocr, inventory_config)
    for page_hocr in pages_hocr:
        print(page_hocr["page_num"], page_parser.get_page_type(page_hocr, inventory_config, debug=False))
        page_hocr["page_type"] += [page_parser.get_page_type(page_hocr, inventory_config, debug=False)]
        for column in page_hocr["columns"]:
            del column["lines"]
        #print(json.dumps(page_hocr, indent=2))

{'year': 1712, 'inventory_num': 3767, 'base_dir': '/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr', 'page_index': 'republic_hocr_pages', 'page_doc_type': 'page', 'scan_index': 'republic_hocr_scans', 'scan_doc_type': 'scan', 'paragraph_index': 'republic_paragraphs', 'paragraph_doc_type': 'paragraph', 'tiny_word_width': 15, 'avg_char_width': 20, 'remove_tiny_words': True, 'remove_line_numbers': False, 'normal_scan_width': 4840, 'column_gap': {'gap_threshold': 50, 'gap_pixel_freq_ratio': 0.75}, 'word_conf_threshold': 10, 'fulltext_char_threshold': 0.5, 'filter_words': ['|', '{', '$', '}'], 'index_page': {'left_jump_ratio_min': 0.5}, 'resolution_page': {'left_jump_ratio_max': 0.3}, 'respect_page': {'column_min_threshold': 3, 'column_max_threshold': 4, 'capitalized_word_line_ratio': 0.3, 'capital_freq_ratio': 0.5}, 'title_page': {'min_char_width': 40, 'min_word_height': 60, 'min_word_num': 10, 'num_top_half_words': 60, 'num_top_half_chars': 150, 'max_line_width_threshold': 1000, 'large_wor

In [228]:
import republic.parser.republic_page_parser as page_parser
from statistics import median, mean, pstdev
from republic.config.republic_config import base_config, set_config_inventory_num
import re

inventory_num = 3819
data_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr"

inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)
print(inventory_config)
scan_files = file_parser.get_files(inventory_config["data_dir"])
print("Number of scan files:", len(scan_files))

scan_file = scan_files[19]
scan_hocr = page_parser.get_scan_hocr(scan_file, inventory_config)
print(scan_hocr["scan_num"], scan_hocr["scan_type"])
if "double_page" in scan_hocr["scan_type"]:
    pages_hocr = page_parser.parse_double_page_scan(scan_hocr, inventory_config)
    
print("num pages:", len(pages_hocr))

    
# TO DO
# - determine page side and page number (scan * 2 for even scan * 2 - 1 for odd)


print()
for scan_file in scan_files[3:9]:
    scan_hocr = page_parser.get_scan_hocr(scan_file, inventory_config)
    #print(scan_hocr["scan_num"], scan_hocr["scan_type"])
    if "double_page" in scan_hocr["scan_type"]:
        pages_hocr = page_parser.parse_double_page_scan(scan_hocr, inventory_config)
        for page_hocr in pages_hocr:
            page_type = page_parser.get_page_type(page_hocr, inventory_config)
            print(page_hocr["page_num"], page_hocr["page_type"])
            #rep_es.index_page(es, page_hocr, inventory_config)
            




{'year': 1764, 'inventory_num': 3819, 'base_dir': '/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr', 'page_index': 'republic_hocr_pages', 'page_doc_type': 'page', 'scan_index': 'republic_hocr_scans', 'scan_doc_type': 'scan', 'paragraph_index': 'republic_paragraphs', 'paragraph_doc_type': 'paragraph', 'tiny_word_width': 15, 'avg_char_width': 20, 'remove_tiny_words': True, 'remove_line_numbers': False, 'normal_scan_width': 4840, 'column_gap': {'gap_threshold': 50, 'gap_pixel_freq_ratio': 0.75}, 'word_conf_threshold': 10, 'fulltext_char_threshold': 0.5, 'filter_words': ['|', '{', '$', '}'], 'index_page': {'left_jump_ratio_min': 0.5}, 'index_page_early_print': {'page_ref_line_threshold': 10, 'left_jump_ratio_threshold': 0.5, 'num_words_min': 200, 'num_words_max': 600, 'inventory_threshold': 3819}, 'index_page_late_print': {'median_line_width_min': 250, 'median_line_width_max': 400, 'num_words_min': 200, 'num_words_max': 500, 'stdev_line_width_min': 100, 'stdev_line_width_max': 400, 'num_li

In [227]:
import republic.elastic.republic_elasticsearch as rep_es
from republic.config.republic_config import base_config, set_config_inventory_num

es = Elasticsearch()

#rep_es.delete_es_index(base_config["page_index"])
#rep_es.delete_es_index(base_config["scan_index"])

inventory_numbers = [
    #3767,
    #3768,
    #3769,
    #3780,
    #3783,
    #3796,
    #3798,
    3819,
    #3838,
    #3853
]

for inventory_num in inventory_numbers:
    rep_es.parse_inventory(es, inventory_num, base_config, data_dir)



In [214]:
inventory_info = {
    "3767": {
        "title_page": [8, 12, 796, 800, 1502], 
        "index_page": {"from": 1502, "to": 10},
        "resolution_page": {"from": 12, "to": 1543}
    },
    "3768": {
        "title_page": [8, 12, 60, 828, 832], 
        "index_page": {"from": 12, "to": 59},
        "resolution_page": {"from": 60, "to": 1699}
    },
    "3769": {
        "title_page": [8, 12, 52, 752, 756], 
        "index_page": {"from": 12, "to": 50},
        "resolution_page": {"from": 52, "to": 1349}
    },
    "3780": {
        "title_page": [9, 11, 90, 565, 567], 
        "index_page": {"from": 11, "to": 89},
        "resolution_page": {"from": 90, "to": 1067}
    },
    "3783": {
        "title_page": [8, 10, 102, 612, 614], 
        "index_page": {"from": 10, "to": 100},
        "resolution_page": {"from": 102, "to": 1063}
    },
    "3796": {
        "title_page": [8, 10, 14, 16, 124, 552, 554], # 16 is a duplicate of 14
        "respect_page": {"from": 10, "to": 13},
        "index_page": {"from": 14, "to": 123},
        "resolution_page": {"from": 124, "to": 997}
    },
    "3798": {
        "title_page": [8, 10, 14, 94, 424, 426],
        "respect_page": {"from": 10, "to": 13},
        "index_page": {"from": 14, "to": 92},
        "resolution_page": {"from": 94, "to": 763}
    },
    "3819": {
        "title_page": [6, 8, 14, 164, 596, 598], 
        "respect_page": {"from": 8, "to": 13},
        "index_page": {"from": 14, "to": 163},
        "resolution_page": {"from": 164, "to": 1026}                
    },
    "3838": {
        "title_page": [6, 8], 
        "resolution_page": {"from": 8, "to": 586}                
    },
    "3853": {
        "title_page": [6, 8, 16, 174,624, 626], 
        "respect_page": {"from": 8, "to": 14},
        "index_page": {"from": 16, "to": 173},
        "resolution_page": {"from": 174, "to": 1080}                
    },
}

## Finding Page Type Sequences

Index, respect and resolution pages come in sequences, e.g. all index pages are in a single sequence without interruption. To detect incorrectly identified page type labels, look for a clusters of pages of a certain type. If most pages in a sequence below to a single type, the few others in between are probably misclassified.

Also, pages of a certain type start with a title page, and are followed by a title page starting a sequence of differnt page type (except for resolution pages that have two title pages halfway through.

In [45]:
from republic.config.republic_config import base_config, set_config_inventory_num
inventory_num = 3853
data_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr"

inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)
print(inventory_config)
scan_files = file_parser.get_files(inventory_config["data_dir"])
print("Number of scan files:", len(scan_files))



# scan num 751 is index start
# scan num 6 is resolution start

scan_file = scan_files[7]
print(scan_file)
scan_hocr = page_parser.get_scan_hocr(scan_file, inventory_config)
#print(scan_hocr)
if "double_page" in scan_hocr["scan_type"]:
    pages_hocr = page_parser.parse_double_page_scan(scan_hocr, inventory_config)
    for page_hocr in pages_hocr:
        print(page_hocr["page_num"], page_hocr["page_type"], page_hocr["num_words"])
        rep_es.index_page(es, page_hocr, inventory_config)
        for column in page_hocr["columns"]:
            del column["lines"]
        #print(json.dumps(page_hocr, indent=2))

{'year': 1790, 'inventory_num': 3853, 'base_dir': '/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr', 'page_index': 'republic_hocr_pages', 'page_doc_type': 'page', 'scan_index': 'republic_hocr_scans', 'scan_doc_type': 'scan', 'paragraph_index': 'republic_paragraphs', 'paragraph_doc_type': 'paragraph', 'tiny_word_width': 15, 'avg_char_width': 20, 'remove_tiny_words': True, 'remove_line_numbers': False, 'normal_scan_width': 4840, 'column_gap': {'gap_threshold': 50, 'gap_pixel_freq_ratio': 0.75}, 'word_conf_threshold': 10, 'fulltext_char_threshold': 0.5, 'filter_words': ['|', '{', '$', '}'], 'index_page': {'left_jump_ratio_min': 0.5}, 'resolution_page': {'left_jump_ratio_max': 0.3}, 'title_page': {'min_char_width': 40, 'min_word_height': 60, 'min_word_num': 10, 'num_top_half_words': 60, 'num_top_half_chars': 150, 'max_line_width_threshold': 1000, 'large_word_lines_threshold': 2, 'title_line_top_threshold': 1300, 'max_word_num': 500}, 'data_dir': '/Users/marijnkoolen/Data/Projects/REPUBLIC/

## Evaluate Title Page Identification

In [215]:
import republic.parser.republic_index_page_parser as index_parser
import republic.elastic.republic_elasticsearch as rep_es
import republic.analyser.republic_inventory_analyser as inv_analyser
import republic.parser.republic_base_page_parser as base_parser
from republic.config.republic_config import base_config, set_config_inventory_num
from elasticsearch import Elasticsearch
from collections import defaultdict

from typing import Union, List

es = Elasticsearch()
data_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr"


inventory_numbers = [
    3767,
    3768,
    3769,
    3780,
    3783,
    3796,
    3798,
    3819,
    3838,
    3853
]

inventory_num = 3767
page_type = "title_page"
page_type = "index_page"
#page_type = "resolution_page"

word_num_threshold = 500


def get_inventory_summary(inventory_config):
    inventory_data = {}
    title_pages = rep_es.retrieve_title_pages(es, inventory_num, inventory_config)
    title_pages.sort(key= lambda x: x["page_num"])
    inventory_data["title_pages"] = title_pages
    inventory_data["title_page_nums"] = [title_page["page_num"] for title_page in title_pages]
    inventory_data["num_pages"] = get_inventory_num_pages(es, inventory_num, inventory_config)
    inventory_data["page_type_sequences"] = inv_analyser.find_page_type_sequences(es, inventory_num, inventory_config)
    #inventory_data["page_type_sequences"] = page_type_sequences

inventory_data = defaultdict(dict)

for inventory_num in inventory_numbers:
    inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)
    title_pages = rep_es.retrieve_title_pages(es, inventory_num, inventory_config)
    title_pages.sort(key= lambda x: x["page_num"])
    inventory_data[inventory_num]["title_pages"] = title_pages
    inventory_data[inventory_num]["title_page_nums"] = [title_page["page_num"] for title_page in title_pages]
    inventory_data[inventory_num]["num_pages"] = get_inventory_num_pages(es, inventory_num, inventory_config)
    should_find = set(inventory_info[str(inventory_num)]["title_page"])
    correct = set()
    for title_page in title_pages:
        if title_page["page_num"] in should_find:
            correct.add(title_page["page_num"])
        #print(inventory_num, title_page["page_num"], title_page["num_words"], title_page["page_type"])
    precision = 0.0
    recall = 0.0
    if len(title_pages):
        precision = len(correct) / len(title_pages)
    recall = len(correct) / len(should_find)
    print(inventory_num, len(correct), len(should_find), precision, recall)
    print("\t", [title_page["page_num"] for title_page in title_pages], should_find)
    page_type_sequences = inv_analyser.find_page_type_sequences(es, inventory_num, inventory_config)
    inventory_data[inventory_num]["page_type_sequences"] = page_type_sequences
    for page_type_sequence in page_type_sequences:
        print("\t", page_type_sequence)
    print()
    #type_order_info = inv_analyser.find_page_type_order(es, inventory_num, inventory_config)
    #print(type_order_info)


3767 5 5 1.0 1.0
	 [8, 12, 796, 800, 1502] {800, 8, 12, 796, 1502}
	 {'start': 1502, 'end': 1541, 'page_type': 'index_page'}
	 {'start': 4, 'end': 1500, 'page_type': 'resolution_page'}

3768 5 5 1.0 1.0
	 [8, 12, 60, 828, 832] {832, 828, 8, 12, 60}
	 {'start': 12, 'end': 59, 'page_type': 'index_page'}
	 {'start': 60, 'end': 1704, 'page_type': 'resolution_page'}

3769 5 5 1.0 1.0
	 [8, 12, 52, 752, 756] {8, 12, 752, 756, 52}
	 {'start': 12, 'end': 50, 'page_type': 'index_page'}
	 {'start': 52, 'end': 1352, 'page_type': 'resolution_page'}

3780 5 5 1.0 1.0
	 [9, 11, 90, 565, 567] {9, 11, 565, 567, 90}
	 {'start': 11, 'end': 89, 'page_type': 'index_page'}
	 {'start': 90, 'end': 1076, 'page_type': 'resolution_page'}

3783 5 5 1.0 1.0
	 [8, 10, 102, 612, 614] {612, 614, 102, 8, 10}
	 {'start': 10, 'end': 107, 'page_type': 'index_page'}
	 {'start': 102, 'end': 1068, 'page_type': 'resolution_page'}

3796 7 7 1.0 1.0
	 [8, 10, 14, 16, 124, 552, 554] {8, 552, 10, 554, 14, 16, 124}
	 {'start': 3

In [212]:
import republic.analyser.republic_inventory_analyser as inv_analyser
from republic.config.republic_config import base_config, set_config_inventory_num
from elasticsearch import Elasticsearch

inventory_num = 3780
#inventory_num = 3819
#inventory_num = 3796

data_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr"
inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)
es = Elasticsearch()


inventory_data = inv_analyser.get_inventory_summary(es, inventory_config)



In [213]:
print(inventory_data["title_page_nums"])
print(inventory_data["num_pages"])
print(inventory_data["page_type_sequences"])

for section in inventory_data["sections"]:
    print(inventory_num, section)



[9, 11, 90, 565, 567]
1067
[{'start': 11, 'end': 89, 'page_type': 'index_page'}, {'start': 90, 'end': 1076, 'page_type': 'resolution_page'}]
3780 {'start': 9, 'end': 10, 'page_type': 'unknown_page_type'}
3780 {'start': 11, 'end': 89, 'page_type': 'index_page'}
3780 {'start': 90, 'end': 564, 'page_type': 'resolution_page'}
3780 {'start': 567, 'end': 1067, 'page_type': 'resolution_page'}


## Check duplicate pages


In [39]:
import republic.elastic.republic_page_checks as page_checker
import republic.elastic.republic_elasticsearch as rep_es

inventory_num = 3796
inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)

page1 = rep_es.retrieve_page_by_page_number(es, 14, inventory_config)
page2 = rep_es.retrieve_page_by_page_number(es, 16, inventory_config)

lines1 = [line["line_text"] for column in page1["columns"] for line in column["lines"]]
lines2 = [line["line_text"] for column in page2["columns"] for line in column["lines"]]

sim = page_checker.compute_page_similarity(page2, page1)
print("sim:", sim, "\n\n")

text1 = " ".join(lines1)
text2 = " ".join(lines2)
dist = page_checker.score_levenshtein_distance(text2, text1)
sim = dist / min([len(text1), len(text2)])
print("sim:", sim, "\n\n")

print(len(lines1))
print(len(lines2))
for index1, line1 in enumerate(lines1):
    print(line1, "\t", lines2[index1])
#for inventory_num in inventory_numbers:
#    inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)
#    page_checker.detect_duplicate_scans(es, inventory_config)


sim: 0.445 


sim: 0.38478747203579416 


83
85
Ï N D 	 I N
D FER 	 DE
R FE S © L U 	 R EF S O L U
V A N 	 VAN
HoocH MoGENDE HEEREN 	 H M
S T A A 1 IN G EÉ 	 OOGH OGEND E EERE N
D ER 	 5
VEREENIGHDE NEDERLANDEN. 	 A A Ï ID N G E N EÉ R A A L
G 	 DER
enoomen in den jaare 1741. 	 V E R E EN IG H D N E D E R L A N D É N.
A. 	 Genoomen 1n den Jaare 1741.
van 	 A
NA 17 advertentie. 473. 	 Ank
MAY IVe Aarnout Pasport ad 	 NY a TE 7
WINE: ‚omnes Populos. 	 A; Gt ad
4 WE Abrahams O&roy om 	 2% PA
7e difponceren. 606. 	 TNG te dij pay 0
EN Admiraliteyten in het 	 SUA a
RIET gemeen, 704 Amfler- 	 SIT EER ij
dam kennijfe geeven- 	 om Henn 9 mr
de van het uytloopen van ’s Lands Schee- 	 de van het uytloopen van ’s 66
pen onder de Capiteynen Lynflager en Lin- 	 pen onder de Capiteynen ee an
claan na Curacao, en reedenen waarom bet 	 claan na Curacao, en reedenen waarom bet
Schip van Capiteyn Panhuys gedeftineert na 	 ED, Capiteyn Panhuys gede na
de Middelandfche Zee, nogh niet in Zee 	 ae Zee, nogh

In [220]:

import republic.elastic.republic_page_checks as page_checker
import republic.elastic.republic_elasticsearch as rep_es

inventory_num = 3796
data_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr"
inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)
es = Elasticsearch()

page_checker.correct_page_types(es, inventory_config)

Gathering section information
section respect_page (10-13)	corrected types of 0 pages
section respect_page (14-15)	corrected types of 1 pages
section index_page (16-123)	corrected types of 10 pages
section resolution_page (124-551)	corrected types of 40 pages
section resolution_page (554-1000)	corrected types of 37 pages

Done!


## Correct page types

In [226]:
data_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr"
es = Elasticsearch()

for inventory_num in inventory_numbers:
    inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)
    page_checker.correct_page_types(es, inventory_config)


Gathering section information for inventory 3767
no document for page number 403
no document for page number 404
no document for page number 405
no document for page number 406
Inventory 3767 section resolution_page (12-795)	corrected types of 40 pages
Inventory 3767 section resolution_page (800-1501)	corrected types of 66 pages
Inventory 3767 section index_page (1502-1540)	corrected types of 0 pages

Done!
Gathering section information for inventory 3768
Inventory 3768 section index_page (12-59)	corrected types of 0 pages
Inventory 3768 section resolution_page (60-827)	corrected types of 57 pages
Inventory 3768 section resolution_page (832-1702)	corrected types of 74 pages

Done!
Gathering section information for inventory 3769
Inventory 3769 section index_page (12-51)	corrected types of 1 pages
Inventory 3769 section resolution_page (52-751)	corrected types of 56 pages
Inventory 3769 section resolution_page (756-1352)	corrected types of 49 pages

Done!
Gathering section information f